In [ ]:
!pip install fastai==0.7.0

In [ ]:
from fastai.imports import *
from fastai.structured import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import os, gc

In [ ]:
#set paths for training and test data
TRAIN_PATH = "../input/microsoft-train"
TEST_PATH = "../input/microsoft/microsoft-malware-prediction"

!ls {TRAIN_PATH}
!ls {TEST_PATH}

In [ ]:
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
#             print("******************************")
#             print("Column: ",col)
#             print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
#             print("dtype after: ",props[col].dtype)
#             print("******************************")
    
    # Print final result
#     print("___MEMORY USAGE AFTER COMPLETION:___")
#     mem_usg = props.memory_usage().sum() / 1024**2 
#     print("Memory usage is: ",mem_usg," MB")
#     print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props

In [ ]:
%%time

training_df = pd.read_csv(f'{TRAIN_PATH}/train.csv', low_memory=False)

# downcast unnecessarily large dtypes to reduce overall size of dataframe
training_df = reduce_mem_usage(training_df)
train_cats(training_df)

# change the HasDetection column's datatype from Float64 to Category
training_df['HasDetections'] = training_df['HasDetections'].astype('category')

In [ ]:
%%time

# clean the dataset by replacing na values, converting category types to numeric types, and stripping 
# the target column from the dataset
df, y, nas = proc_df(training_df, 'HasDetections', subset=400_000)

#delete the original training dataset; no longer need it
#del training_df
#gc.collect()

In [ ]:
# function to split training set to set aside a validation set
def split_data(data, n):
    return data[:n].copy(), data[n:].copy()

# create validation set
n_valid = 100_000
n_trn = len(df)-n_valid

X_train, X_valid = split_data(df, n_trn)
y_train, y_valid = split_data(y, n_trn)

In [ ]:
%%time 

#train the Random Forest classifier model
clf = RandomForestClassifier(n_estimators=50, n_jobs=-1, verbose=1)
clf.fit(X_train, y_train)

#shows the importance of each feature as a predictor of whether a given machine has any
#malware detections
print(clf.feature_importances_)

prediction = clf.predict(X_valid)

# how'd we do?
roc_auc_score(y_valid, prediction)

In [ ]:
%%time 

# do some of that good, good garbage collection
del X_train, X_valid, y_train, y_valid, df, y 
gc.collect()

#create DataFrame to store results in
results_cols = ['MachineIdentifier','HasDetections']
results_df = pd.DataFrame(columns=results_cols)

ranges = [None, range(1,1_000_000), range(1, 2_000_000), range(1, 3_000_000), 
          range(1, 4_000_000), range(1, 5_000_000), range(1, 6_000_000), 
          range(1, 7_000_000)]

# read in the test set and do all the processing steps
for i in ranges:
    test_chunk = pd.read_csv(f'{TEST_PATH}/test.csv', low_memory=False, skiprows=i, nrows=1_000_000)
    
    # clean the dataset
    test_chunk = reduce_mem_usage(test_chunk)
    train_cats(test_chunk)
    test_df, _, nas = proc_df(test_chunk, na_dict=nas)
    
    # make prediction for the chunk 
    predictions = clf.predict(test_df)

    # create a temporary DataFrame to store current iterations predictions and append results to global list
    tmp_df = pd.DataFrame({'MachineIdentifier': test_chunk['MachineIdentifier'],
                         'HasDetections': predictions})
    results_df = results_df.append(tmp_df)
    
    # delete everything from this iteration
    del tmp_df, test_df, test_chunk
    gc.collect()
    
# reset row indices 
results_df = results_df.reset_index(drop=True)

In [ ]:
# drop duplicate rows 
results_copy = results_df.drop_duplicates(subset='MachineIdentifier', keep='first', inplace=False)
del results_df
gc.collect()

# save results to csv file
results_copy.to_csv('malware_predictions.csv', index=False)